In [ ]:
import pandas as pd
import re
import numpy as np
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

: 

In [4]:
path = 'GL.xlsx'
df = pd.read_excel(path, sheet_name=0)
df.head()

,Company Code,Posting period,Ledger,Reference,Document type,Document Header Text,Document Number,Text,Document Currency Key,Document Currency Value,...,Reversal Reason,Reversed With,Reverse Posting Date,Entry Date,Parked By,User Name,Tax Code ( MWSK1 ),Tax code ( MWSKZ ),EY Classification,EY treaments
0,4690,3,G8,PAYROLL MAR22,SJ,Payroll Mar 2022,3100001999,Basic_Salary,VND,-670549348.0,...,1.0,3.100002e+09,NaT,2022-03-31,NaN,N0224528,NaN,NaN,Basic Salary,Deductible
1,4690,3,G8,PAYROLL MAR22,SJ,Payroll Mar 2022,3100001999,Basic_Salary,VND,-363164652.0,...,1.0,3.100002e+09,NaT,2022-03-31,NaN,N0224528,NaN,NaN,Basic Salary,Deductible
2,4690,3,G8,PAYROLL MAR22,SJ,Payroll Mar 2022,3100001999,Basic_Salary,VND,-79680000.0,...,1.0,3.100002e+09,NaT,2022-03-31,NaN,N0224528,NaN,NaN,Basic Salary,Deductible
3,4690,3,G8,PAYROLL MAR22,SJ,Payroll Mar 2022,3100001999,Basic_Salary,VND,-20583000.0,...,1.0,3.100002e+09,NaT,2022-03-31,NaN,N0224528,NaN,NaN,Basic Salary,Deductible
4,4690,3,G8,PAYROLL MAR22,SJ,Payroll Mar 2022,3100001999,Basic_Salary,VND,-18417000.0,...,1.0,3.100002e+09,NaT,2022-03-31,NaN,N0224528,NaN,NaN,Basic Salary,Deductible


In [55]:
kw = pd.read_excel(path, sheet_name=1)
keyword = list(kw['Text content'].str.lower())
keyword_ = list(kw['Text content'])
change_kw = list(kw['Text return'])
keyword

['marketing',
 'network access',
 'tet gift',
 'annual contest',
 'gift',
 'charge?20',
 'gst/vat',
 'consulting ',
 'agent fee',
 'airticket',
 'allocate',
 'office rent',
 'lunch',
 'delivery',
 'car rental',
 'cartridges',
 'catastrophe',
 'cứu hộ',
 'cit',
 'claims',
 'cloud',
 'cuoc',
 'performance cam',
 'media part',
 'lead',
 'office rent',
 'defer tax ',
 'telephone',
 'storage',
 'voucher',
 'earned',
 'electricity',
 'gotit',
 'postage',
 'office lease',
 'mineral water',
 'health insurance',
 'hn21',
 'member fee',
 'internet',
 'khám sức khỏe',
 'kinh phí dự thi đại lý',
 'water',
 'legal service fee ',
 'hotel',
 'allowance',
 'dinner',
 'lunch',
 'gasoline',
 'medical check-up',
 'medical consultant',
 'office cleaning',
 'office plant display',
 'mainternance',
 'cuoc kenh truyen',
 'opex',
 'pantry',
 'parking',
 'meal',
 'phone',
 'prepaid',
 'settlement',
 'sms',
 'viettel',
 'perdiem',
 'yep']

In [38]:
df1 = df[['Text']].astype(str)
df1['Text'] = df['Text'].str.lower()
df1.fillna('NO_DATA', inplace=True)

In [45]:
def return_keyword(column, key_words):
    for i in key_words:
        if i in column:
            return i[0].upper()+ i[1:]
    return ''

df1['return_kw'] = df1['Text'].apply(return_keyword, key_words= keyword)
df1[df1['return_kw'] != '']

,Text,return_kw
96,prepaid liv pols s-hga-00000123-12-22 apr-dec22,Prepaid
97,prepaid liv pols s-pag-00000321-09-22 apr-dec22,Prepaid
98,health insurance 01apr-31dec22,Health insurance
99,health insurance 01apr-31dec22,Health insurance
100,health insurance 01apr-31dec22,Health insurance
...,...,...
98585,quy 1% gk dealer cl voucher reward t.chinh,Voucher
98586,quy 1% gk dealer cl voucher reward,Voucher
98588,quy 1% gk dealer cl voucher reward,Voucher
98589,quy 1% gk dealer cl voucher reward t.thuan,Voucher


In [70]:
new_dict = {k: v for k, v in zip(keyword_, change_kw)}
df2 = df1.replace({'return_keyword': new_dict})
df2.tail()

,Text,return_kw
99473,NO_DATA,
99474,NO_DATA,
99475,NO_DATA,
99476,NO_DATA,
99477,interest for late payment for aug+sep21,


In [95]:
df_return = pd.concat([df, df2], axis=1)
df_return.shape
df_return[df_return['return_kw'] != '']

,Company Code,Posting period,Ledger,Reference,Document type,Document Header Text,Document Number,Text,Document Currency Key,Document Currency Value,...,Reverse Posting Date,Entry Date,Parked By,User Name,Tax Code ( MWSK1 ),Tax code ( MWSKZ ),EY Classification,EY treaments,Text,return_kw
96,4690,3,G8,PREPAID HGA,XB,Prepaid INSURANCE,7000001530,Prepaid LIV Pols S-HGA-00000123-12-22 Apr-Dec22,VND,-170818425.0,...,NaT,2022-03-28,NaN,N0192146,NaN,VB,NaN,NaN,prepaid liv pols s-hga-00000123-12-22 apr-dec22,Prepaid
97,4690,3,G8,PREPAID PAG,XB,Prepaid INSURANCE,7000001531,Prepaid LIV Pols S-PAG-00000321-09-22 Apr-Dec22,VND,-119507693.0,...,NaT,2022-03-28,NaN,N0192146,NaN,VB,NaN,NaN,prepaid liv pols s-pag-00000321-09-22 apr-dec22,Prepaid
98,4690,3,G8,PREPAID BAO VIET,XB,Prepaid INSURANCE,7000001532,Health insurance 01Apr-31Dec22,VND,-136781150.0,...,NaT,2022-03-28,NaN,N0192146,NaN,VB,NaN,NaN,health insurance 01apr-31dec22,Health insurance
99,4690,3,G8,PREPAID BAO VIET,XB,Prepaid INSURANCE,7000001532,Health insurance 01Apr-31Dec22,VND,-82875228.0,...,NaT,2022-03-28,NaN,N0192146,NaN,VB,NaN,NaN,health insurance 01apr-31dec22,Health insurance
100,4690,3,G8,PREPAID BAO VIET,XB,Prepaid INSURANCE,7000001532,Health insurance 01Apr-31Dec22,VND,-19402021.0,...,NaT,2022-03-28,NaN,N0192146,NaN,VB,NaN,NaN,health insurance 01apr-31dec22,Health insurance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98585,4690,1,G8,ACCRUAL BD,XA,Accrual BD-Phan Trang,7000001369,Quy 1% GK Dealer CL Voucher reward T.Chinh,VND,156300000.0,...,NaT,2022-01-24,NaN,N0224528,NaN,VB,NaN,NaN,quy 1% gk dealer cl voucher reward t.chinh,Voucher
98586,4690,1,G8,ACCRUAL DO DUNG,AC,ACCRUAL DO DUNG,100001711,Quy 1% GK Dealer CL Voucher reward,VND,-154900000.0,...,NaT,2022-01-01,NaN,BATCH_BASIS,NaN,VB,NaN,NaN,quy 1% gk dealer cl voucher reward,Voucher
98588,4690,1,G8,ACCRUAL DO DUNG,AC,ACCRUAL DO DUNG,100001711,Quy 1% GK Dealer CL Voucher reward,VND,-514000000.0,...,NaT,2022-01-01,NaN,BATCH_BASIS,NaN,VB,NaN,NaN,quy 1% gk dealer cl voucher reward,Voucher
98589,4690,1,G8,ACCRUAL BD,XA,Accrual BD-Phan Trang,7000001369,Quy 1% GK Dealer CL Voucher reward T.Thuan,VND,19800000.0,...,NaT,2022-01-24,NaN,N0224528,NaN,VB,NaN,NaN,quy 1% gk dealer cl voucher reward t.thuan,Voucher


In [97]:
df_return.to_csv('GL_return.csv', index=False, encoding='utf8')

In [4]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession 
spark = SparkSession.builder.appName("Python Spark DataFrames basic example").config("spark.some.config.option", "some-value").getOrCreate()

df = spark.read.option('header', True).csv('GL_return.csv', )
df.show()

AnalysisException: Cannot resolve column name "Text" among (Company Code, Posting period, Ledger, Reference, Document type, Document Header Text, Document Number, Text7, Document Currency Key, Document Currency Value, Company Code Currency Key, Company Code Currency Value, G/L Account, G/L Account: Long Text, Country General Ledger Account, Country G/L Account Description, Reporting Acct, Reporting Account Description, Business Area, Profit Center, Profit Center: Long Text, Channel, Channel Description, Cost Center, Cost Center: Long Text, RI Code, RICODE Description, Risk Location, Risk Location Description, Code Block 1, Code Block 1 Description, Trading partner, Und YR, CYPY, Acc YR, Cat Code, Catastrophe Code Description, Transaction type, Group Currency Key, Group Currency Value, Global Company Currency Key, Global Company Currency Value, Reversal Reason, Reversed With, Reverse Posting Date, Entry Date, Parked By, User Name, Tax Code ( MWSK1 ), Tax code ( MWSKZ ), EY Classificatio

(99478, 2)